In [ ]:
Author: Sorya Ek

In this assignment, we will be doing pathfinding using Dijkstra's and A* to search through the topology.

In this assignment we will:

* Parse a data file to create a representation of a world-space
* Implement functions that operate over this representation: telling the algorithms how to navigate this space, how to estimate costs over this space, and how to determine when a goal has been reached in this space
* Implement Dijkstra's (an algorithm for finding the optimal path through a graph) search and A* search (a modification of Dijkstra's that utilizes heuristics to speed up the search, while still guaranteeing optimality)


First we will load the map into a grid called: terrain

In [23]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

import pandas as pd
import io
df = io.StringIO(uploaded['terrain.txt'].decode('utf-8'))
terrain = [list(row.rstrip()) for row in df]

Saving terrain.txt to terrain (2).txt
User uploaded file "terrain.txt" with length 489 bytes


In [24]:
for row in terrain:
    print(row)

['😀', '🌿', '🌿', '🌿', '🌿', '🌼', '🌿', '🌼', '🌼', '🌿', '🌿', '🌿']
['🌿', '🌿', '🌿', '🌿', '🌿', '🌼', '🌿', '🌼', '🌼', '🌿', '🌿', '🌿']
['🌿', '🌿', '🌿', '🌿', '🌿', '🌿', '🌼', '🌿', '🌿', '🌿', '🌿', '🌿']
['🌿', '🌿', '🌿', '🌊', '🌊', '🌊', '🌊', '🌊', '🌉', '🌊', '🌊', '🌊']
['🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌉', '🌊', '🌊', '🌊']
['🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌉', '🌊', '🌊', '🌊']
['🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌉', '🌊', '🌊', '🌊']
['🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌉', '🌊', '🌊', '🌊']
['🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌊', '🌉', '🌊', '🌊', '🌊']
['🌿', '🌿', '🌿', '🌲', '🌿', '🌿', '🌿', '🌿', '🌿', '🌼', '🌲', '🌲']


The indexing on terrain is terrain[y][x].

Now we will implement a find_neighbors function. find_neighbors should take in the curent position (a tuple of (x,y)) and the terrain. It will output a list [] of all of the neighbors (tuples of ( (x,y), cost)) the costs are as follows: 🌿 = 1 🌼 = 2 🌉 = 1 🌊 = 5 🌲 = 1

i.e., we are fine walking on grass, bridges, and trees, but would prefer to avoid flowers, and really don't want to swim.

Note: this is assuming a neighborhood of:

🌿🌿🌿

🌿🌿🌿

🌿🌿🌿

not

  🌿 

🌿🌿🌿

  🌿 

i.e. We are allowed to move on diagonals, not just in the cardinal directions

In [0]:
def find_neighbors(current_position, terrain):
    
    num_cols = len(terrain[0])
    num_rows = len(terrain)
    
    x = current_position[0]
    y = current_position[1]
    
    neighbors = []
    
    if x==0:
      if y==0:
        neighbors.append(((x+1, y+1), get_cost((x+1, y+1), terrain)))
        neighbors.append(((x+1, y), get_cost((x+1,y), terrain)))
        neighbors.append(((x, y+1), get_cost((x, y+1), terrain)))
      elif y == num_cols-1:
        neighbors.append(((x, y-1), get_cost((x, y-1), terrain))) 
        neighbors.append(((x+1, y), get_cost((x+1, y), terrain)))
        neighbors.append(((x+1, y-1), get_cost((x+1, y-1), terrain))) 
      else:
        neighbors.append(((x+1, y+1), get_cost((x+1, y+1), terrain)))
        neighbors.append(((x, y+1), get_cost((x, y+1), terrain))) 
        neighbors.append(((x+1, y), get_cost((x+1, y), terrain)))        
        neighbors.append(((x, y-1), get_cost((x, y-1), terrain))) 
        neighbors.append(((x+1, y-1), get_cost((x+1, y-1), terrain)))
    
    elif x==num_rows-1:
      if y==0:
        neighbors.append(((x, y+1), get_cost((x, y+1), terrain))) 
        neighbors.append(((x-1, y), get_cost((x-1, y), terrain)))
        neighbors.append(((x-1, y+1), get_cost((x-1, y+1), terrain)))
      elif y == num_cols-1:
        neighbors.append(((x, y-1), get_cost((x, y-1), terrain))) 
        neighbors.append(((x-1, y), get_cost((x-1, y), terrain)))
        neighbors.append(((x-1, y-1), get_cost((x-1, y-1), terrain))) 
      else:
        neighbors.append(((x, y+1), get_cost((x, y+1), terrain))) 
        neighbors.append(((x-1, y), get_cost((x-1, y), terrain)))
        neighbors.append(((x-1, y+1), get_cost((x-1, y+1), terrain)))
        neighbors.append(((x, y-1), get_cost((x, y-1), terrain))) 
        neighbors.append(((x-1, y-1), get_cost((x-1, y-1), terrain)))
    else:
      if y==0:
        neighbors.append(((x, y+1), get_cost((x, y+1), terrain))) 
        neighbors.append(((x+1, y), get_cost((x+1, y), terrain)))
        neighbors.append(((x+1, y+1), get_cost((x+1, y+1), terrain)))        
        neighbors.append(((x-1, y), get_cost((x-1, y), terrain)))
        neighbors.append(((x-1, y+1), get_cost((x-1, y+1), terrain)))
      elif y == num_cols-1:
        neighbors.append(((x, y-1), get_cost((x, y-1), terrain))) 
        neighbors.append(((x+1, y), get_cost((x+1, y), terrain)))
        neighbors.append(((x+1, y-1), get_cost((x+1, y-1), terrain)))
        neighbors.append(((x-1, y), get_cost((x-1, y), terrain)))
        neighbors.append(((x-1, y-1), get_cost((x-1, y-1), terrain))) 
      else:
        neighbors.append(((x+1, y+1), get_cost((x+1, y+1), terrain)))
        neighbors.append(((x, y+1), get_cost((x, y+1), terrain))) 
        neighbors.append(((x-1, y), get_cost((x-1, y), terrain)))
        neighbors.append(((x-1, y+1), get_cost((x-1, y+1), terrain)))
        neighbors.append(((x, y-1), get_cost((x, y-1), terrain))) 
        neighbors.append(((x-1, y-1), get_cost((x-1, y-1), terrain)))
        neighbors.append(((x+1, y), get_cost((x+1, y), terrain)))      
        neighbors.append(((x+1, y-1), get_cost((x+1, y-1), terrain)))
    
    return neighbors

In [0]:
def get_cost(position, terrain):
    x = position[0]
    y = position[1]
    if terrain[x][y] == '🌿':
        return 1
    elif terrain[x][y] == '🌼':
        return 2
    elif terrain[x][y] == '🌉':
        return 1
    elif terrain[x][y] == '🌊':
        return 5
    elif terrain[x][y] == '🌲':
        return 1
    elif terrain[x][y] == '😀':
        return 0
    else:
        return np.Inf
        

Now, we want to find the heuristic cost for a given location on the terrain.

Find the Manhattan distance to the nearest 🌲 -- e.g. if the tree is at (x',y') and the given location is (x,y) the heuristic distance is abs(y-y') + abs(x-x')

In [0]:
def get_heuristic(position,terrain):
    min_distance = 0
    num_r = len(terrain)
    num_c = len(terrain[0])
    for i in range(num_r):
      for j in range(num_c):
        if i != position[0] and j != position[1]:
          if is_goal((i,j), terrain):
            dist = abs(position[1]-j) + abs(position[0]-i)
            if dist < min_distance:
              min_distance = dist
    return min_distance

We also need to implement a function that recognizes whether the goal state has been achieved, i.e. is there a 🌲 found at the current position.

In [0]:
def is_goal(position, terrain):
    #fill this in
    return terrain[position[0]][position[1]] == '🌲'

Finally, here is a helper class -- PriorityQueue -- and a helper function pretty_print_path that takes in the path (a list of position (x,y) tuples) and outputs a pretty string with emoji showing the path through the terrain

In [0]:
import heapq

class PriorityQueue:
    def __init__(self):
        self.elements = []
        self.visited = 0
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, self.visited, item))
        self.visited += 1
    def get(self):
        return heapq.heappop(self.elements)[2]
    
def pretty_print_path(path,terrain):
        
    emojis = ['😀','😁','😂','🤣','😃','😄','😅','😆','😉','😊','😋']
    
    path2len = {location:distance for distance,location in enumerate(path)}
    output = []
    for yy,row in enumerate(terrain):
        row_str = ''
        for xx, cur in enumerate(row):
            if (yy, xx) in path2len:
                row_str += emojis[path2len[(yy,xx)] % len(emojis)]
            else:
                row_str += cur
        output.append(row_str)
    return '\n'.join(output)

Now implement Dijkstra's and A*. Each function should return a path (list of tuples (x,y) pairs going from start to end) for the path.


In [0]:
import numpy as np
def dijkstras(initial_position,world,get_neighbors,is_goal): 
    Q = PriorityQueue()
    dist = {}
    visited = {}
    visited_set = set([initial_position])
    p = {}
    Qd = {}    
    dist[initial_position] = 0
    
    for v in get_neighbors(initial_position, world):
      dist[v[0]] = get_cost(v[0], world)
      item = [dist[v[0]], initial_position, v[0]]
      num_items = Q.visited
      Q.put(item, dist[v[0]])
      Qd[v[0]] = (dist[v[0]], num_items, item)    
    
    count = 0
    while not Q.empty():
      q = Q.get()
      u = q[2]
      if u not in visited_set:
        p[u] = q[1]
        visited_set.add(u)
        #visited[i[0]] = True
        if is_goal(u, world):
           a = u
           path_to_goal = [] 
           while a != initial_position:
               path_to_goal.append(a)
               a = p[a]
           print("Number of visited nodes: ", Q.visited)   
           return path_to_goal

        for v in get_neighbors(u, world):
          if dist.get(v[0]):
            if dist[v[0]] > (v[1] + dist[u]):
              dist[v[0]] = v[1] + dist[u]
              t = Qd[v[0]]
              lst = list(t)
              lst[2][0] = dist[v[0]]
              lst[2][1] = u
              t = tuple(lst)
              Qd[v[0]] = t
              heapq._siftdown(Q.elements, 0, Q.elements.index(Qd[v[0]]))
          else:
            dist[v[0]] = v[1] + dist[u]
            item = [dist[v[0]], u, v[0]]
            num_items = Q.visited
            Q.put(item, dist[v[0]])
            Qd[v[0]] = (dist[v[0]], num_items, item) 
   
  
    return []

def a_star(initial_position,world,get_neighbors,is_goal,heuristic):
    Q = PriorityQueue()
    came_from = {}
    cost_so_far = {}
    came_from[initial_position] = None
    cost_so_far[initial_position] = 0
    Q.put(initial_position, 0)
    while not Q.empty():
      u = Q.get()
      if is_goal(u, world):
        a = u
        path_to_goal = [] 
        while a != initial_position:
          path_to_goal.append(a)
          a = came_from[a] 
        print("Number of visited nodes: ", Q.visited)
        return path_to_goal
      for v in get_neighbors(u, world):
        new_cost = cost_so_far[u] + get_cost(v[0], world)
        if v[0] not in cost_so_far or new_cost < cost_so_far[v[0]]:
          cost_so_far[v[0]] = new_cost
          priority = new_cost + heuristic(v[0], world)
          Q.put(v[0], priority)
          came_from[v[0]] = u
    
    return []

In [31]:
print(pretty_print_path(dijkstras((0,0), terrain, find_neighbors, is_goal), terrain))

Number of visited nodes:  99
😀🌿🌿🌿🌿🌼🌿🌼🌼🌿🌿🌿
🌿🤣🌿🌿🌿🌼😊🌼🌼🌿🌿🌿
🌿🌿😂😁😀😋🌼😉🌿🌿🌿🌿
🌿🌿🌿🌊🌊🌊🌊🌊😆🌊🌊🌊
🌊🌊🌊🌊🌊🌊🌊🌊😅🌊🌊🌊
🌊🌊🌊🌊🌊🌊🌊🌊😄🌊🌊🌊
🌊🌊🌊🌊🌊🌊🌊🌊😃🌊🌊🌊
🌊🌊🌊🌊🌊🌊🌊🌊🤣🌊🌊🌊
🌊🌊🌊🌊🌊🌊🌊🌊😂🌊🌊🌊
🌿🌿🌿🌲🌿🌿🌿🌿🌿😁😀🌲


In [32]:
print(pretty_print_path(a_star((0,0), terrain, find_neighbors, is_goal, get_heuristic), terrain))

Number of visited nodes:  99
😀🌿🌿🌿🌿🌼🌿🌼🌼🌿🌿🌿
🌿🤣🌿🌿🌿🌼😊🌼🌼🌿🌿🌿
🌿🌿😂😁😀😋🌼😉🌿🌿🌿🌿
🌿🌿🌿🌊🌊🌊🌊🌊😆🌊🌊🌊
🌊🌊🌊🌊🌊🌊🌊🌊😅🌊🌊🌊
🌊🌊🌊🌊🌊🌊🌊🌊😄🌊🌊🌊
🌊🌊🌊🌊🌊🌊🌊🌊😃🌊🌊🌊
🌊🌊🌊🌊🌊🌊🌊🌊🤣🌊🌊🌊
🌊🌊🌊🌊🌊🌊🌊🌊😂🌊🌊🌊
🌿🌿🌿🌲🌿🌿🌿🌿🌿😁😀🌲
